<a href="https://colab.research.google.com/github/alfonsorinconbarrera/alfonsorinconbarrera/blob/main/Slider_comapracion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
import base64, io
from PIL import Image
import ipywidgets as widgets

# Función para cargar las imágenes y ajustarlas a un tamaño común
def cargar_imagenes():
    print("Selecciona la primera imagen:")
    uploaded_file1 = files.upload()
    file1_name = list(uploaded_file1.keys())[0]

    print("Selecciona la segunda imagen:")
    uploaded_file2 = files.upload()
    file2_name = list(uploaded_file2.keys())[0]

    img1 = Image.open(io.BytesIO(uploaded_file1[file1_name]))
    img2 = Image.open(io.BytesIO(uploaded_file2[file2_name]))

    # Se determina el tamaño común
    width = max(img1.width, img2.width)
    height = max(img1.height, img2.height)

    img1 = img1.resize((width, height))
    img2 = img2.resize((width, height))

    return img1, img2, width, height

# Función para generar fotogramas del GIF simulando el movimiento del deslizador
def crear_frames_gif(img1, img2, frames=20):
    frame_list = []
    width, height = img1.size
    step = max(1, width // frames)

    # Desplazamiento de izquierda a derecha
    for pos in range(0, width + 1, step):
        frame = img1.copy()
        region = img2.crop((0, 0, pos, height))
        frame.paste(region, (0, 0))
        frame_list.append(frame)

    # Regreso de derecha a izquierda
    for pos in range(width, -1, -step):
        frame = img1.copy()
        region = img2.crop((0, 0, pos, height))
        frame.paste(region, (0, 0))
        frame_list.append(frame)

    return frame_list

# Función para guardar el GIF y descargarlo
def guardar_gif(frame_list, duration=100):
    filename = 'comparador_animado.gif'
    frame_list[0].save(filename, save_all=True, append_images=frame_list[1:], duration=duration, loop=0)
    return filename

# Función principal para crear el GIF comparador
def crear_gif_comparador():
    img1, img2, width, height = cargar_imagenes()
    frames = crear_frames_gif(img1, img2, frames=20)
    gif_file = guardar_gif(frames, duration=100)
    print("GIF generado. Se descargará el archivo.")
    files.download(gif_file)

crear_gif_comparador()


Selecciona la primera imagen:


Saving Cocuy2.jpg to Cocuy2.jpg
Selecciona la segunda imagen:


Saving Cocuy1.jpg to Cocuy1.jpg
GIF generado. Se descargará el archivo.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>